In [1]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

- Para esse exemplo irei filtrar as mensagens bulk pq são mensagens de campanha eu quero saber a questão de qual o melhor canal para esse tipo de mensagem

In [2]:
df_msg = pd.read_csv('../dataset/raw/messages-demo.csv')
# df_msg = df_msg[0:1000]
# df_msg = df_msg[df_msg['channel']=='email']
df_msg.head()

C:\Users\Carlos\AppData\Local\Temp\ipykernel_10756\1129225391.py:1: DtypeWarning: Columns (7,8,16,17,19,21,23,25,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_msg = pd.read_csv('../dataset/raw/messages-demo.csv')


,id,message_id,campaign_id,message_type,client_id,channel,category,platform,email_provider,stream,...,is_soft_bounced,soft_bounced_at,is_complained,complained_at,is_blocked,blocked_at,is_purchased,purchased_at,created_at,updated_at
0,3527358,3f6aaad3-bab7-4886-b083-fe8c1f210066,31,transactional,1515915625489833514,email,NaN,NaN,mail.ru,desktop,...,f,NaN,f,NaN,f,NaN,t,2021-05-06 16:40:38,2023-04-27 08:55:05.883908,2023-04-27 08:57:33.080129
1,3527619,0e670ecc-4549-44f6-86ed-469682d34837,32,transactional,1515915625489220445,email,NaN,NaN,yandex.ru,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-27 08:55:06.265821,2023-04-27 08:56:18.60223
2,3527980,276b25cf-1bda-4faf-b3a4-98e4161f9357,32,transactional,1515915625489854185,email,NaN,NaN,mail.ru,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-27 08:55:06.777039,2023-04-27 08:56:19.112546
3,3528369,4545aff2-09b3-45e3-9abd-c680357e5429,32,transactional,1515915625489101550,email,NaN,NaN,mail.ru,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-27 08:55:07.325906,2023-04-27 08:56:19.590637
4,3528648,5850858d-2dcf-4f31-a0d3-5db5649b17c4,32,transactional,1515915625490455948,email,NaN,NaN,mail.ru,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-27 08:55:07.727792,2023-04-27 08:56:19.926474


In [3]:
df_msg["date"] = pd.to_datetime(df_msg["date"], errors="coerce")
df_msg["sent_at"] = pd.to_datetime(df_msg["sent_at"], errors="coerce")
df_msg["clicked_first_time_at"] = pd.to_datetime(df_msg["clicked_first_time_at"], errors="coerce")

In [4]:
cols = ['id', 'message_id', 'campaign_id', 'message_type', 'client_id', 'channel', 'date', 'sent_at', 'is_clicked', 'clicked_first_time_at']
df_msg = df_msg[cols]

In [5]:
df_msg.head()

,id,message_id,campaign_id,message_type,client_id,channel,date,sent_at,is_clicked,clicked_first_time_at
0,3527358,3f6aaad3-bab7-4886-b083-fe8c1f210066,31,transactional,1515915625489833514,email,2021-04-30,2021-04-30 11:27:43,t,2021-05-04 05:47:46
1,3527619,0e670ecc-4549-44f6-86ed-469682d34837,32,transactional,1515915625489220445,email,2021-04-30,2021-04-30 08:00:35,f,NaT
2,3527980,276b25cf-1bda-4faf-b3a4-98e4161f9357,32,transactional,1515915625489854185,email,2021-04-30,2021-04-30 05:56:37,f,NaT
3,3528369,4545aff2-09b3-45e3-9abd-c680357e5429,32,transactional,1515915625489101550,email,2021-04-30,2021-04-30 05:53:59,f,NaT
4,3528648,5850858d-2dcf-4f31-a0d3-5db5649b17c4,32,transactional,1515915625490455948,email,2021-04-30,2021-04-30 06:20:37,f,NaT


# Create Features

In [6]:
## Criar hora, período e dia da semana
df_msg["sending_time"] = df_msg["sent_at"].dt.hour
df_msg["period_day"] = pd.cut(df_msg["sending_time"],
                           bins=[-1, 5, 11, 17, 23],
                           labels=["madrugada", "manha", "tarde", "noite"])
df_msg["day_of_week"] = df_msg["sent_at"].dt.day_name()

In [7]:
df_msg.head()

,id,message_id,campaign_id,message_type,client_id,channel,date,sent_at,is_clicked,clicked_first_time_at,sending_time,period_day,day_of_week
0,3527358,3f6aaad3-bab7-4886-b083-fe8c1f210066,31,transactional,1515915625489833514,email,2021-04-30,2021-04-30 11:27:43,t,2021-05-04 05:47:46,11,manha,Friday
1,3527619,0e670ecc-4549-44f6-86ed-469682d34837,32,transactional,1515915625489220445,email,2021-04-30,2021-04-30 08:00:35,f,NaT,8,manha,Friday
2,3527980,276b25cf-1bda-4faf-b3a4-98e4161f9357,32,transactional,1515915625489854185,email,2021-04-30,2021-04-30 05:56:37,f,NaT,5,madrugada,Friday
3,3528369,4545aff2-09b3-45e3-9abd-c680357e5429,32,transactional,1515915625489101550,email,2021-04-30,2021-04-30 05:53:59,f,NaT,5,madrugada,Friday
4,3528648,5850858d-2dcf-4f31-a0d3-5db5649b17c4,32,transactional,1515915625490455948,email,2021-04-30,2021-04-30 06:20:37,f,NaT,6,manha,Friday


In [8]:
# Tempo do ultimo clique
df_msg = df_msg.sort_values(by=["client_id", "sent_at"])
df_msg["last_click"] = df_msg.groupby("client_id")["clicked_first_time_at"].shift(1)
df_msg["recency_last_click"] = (df_msg["sent_at"] - df_msg["last_click"]).dt.days

In [9]:
df_msg.head()

,id,message_id,campaign_id,message_type,client_id,channel,date,sent_at,is_clicked,clicked_first_time_at,sending_time,period_day,day_of_week,last_click,recency_last_click
13115,18006688,7c48ccee-e2a6-4b50-a5d3-7c1dc8016f12,18011,trigger,1515915625467997301,email,2021-04-30,2021-04-30 09:00:18,f,NaT,9,manha,Friday,NaT,NaN
1160263,18006690,a83c435e-4d05-4a1e-af01-9d67913f371a,18011,trigger,1515915625467997301,email,2021-04-30,2021-04-30 13:06:09,f,NaT,13,tarde,Friday,NaT,NaN
1160262,18006689,fbb0af74-6330-4fc8-a501-06abcc52e257,18010,trigger,1515915625467997301,email,2021-04-30,2021-04-30 16:07:05,f,NaT,16,tarde,Friday,NaT,NaN
1400507,18219075,d533bea7-63d7-4330-a73a-c1ad0b5f876c,19017,trigger,1515915625467997301,email,2021-05-04,2021-05-04 08:27:03,f,NaT,8,manha,Tuesday,NaT,NaN
1400506,18219074,68f1a67b-cb28-4301-95a5-83304ebf8e6b,18011,trigger,1515915625467997301,email,2021-05-04,2021-05-04 09:08:17,f,NaT,9,manha,Tuesday,NaT,NaN


In [10]:
# Segmento Comportamental (Frequencia de Clique)
click_per_customer = df_msg.groupby("client_id")["clicked_first_time_at"].count()
def segmentar(freq):
    if freq == 0:
        return "silencioso"
    elif freq < 3:
        return "ocasional"
    else:
        return "engajador"

df_msg["behavioral_segment"] = df_msg["client_id"].map(click_per_customer).fillna(0).apply(segmentar)

In [11]:
df_msg.head()

,id,message_id,campaign_id,message_type,client_id,channel,date,sent_at,is_clicked,clicked_first_time_at,sending_time,period_day,day_of_week,last_click,recency_last_click,behavioral_segment
13115,18006688,7c48ccee-e2a6-4b50-a5d3-7c1dc8016f12,18011,trigger,1515915625467997301,email,2021-04-30,2021-04-30 09:00:18,f,NaT,9,manha,Friday,NaT,NaN,silencioso
1160263,18006690,a83c435e-4d05-4a1e-af01-9d67913f371a,18011,trigger,1515915625467997301,email,2021-04-30,2021-04-30 13:06:09,f,NaT,13,tarde,Friday,NaT,NaN,silencioso
1160262,18006689,fbb0af74-6330-4fc8-a501-06abcc52e257,18010,trigger,1515915625467997301,email,2021-04-30,2021-04-30 16:07:05,f,NaT,16,tarde,Friday,NaT,NaN,silencioso
1400507,18219075,d533bea7-63d7-4330-a73a-c1ad0b5f876c,19017,trigger,1515915625467997301,email,2021-05-04,2021-05-04 08:27:03,f,NaT,8,manha,Tuesday,NaT,NaN,silencioso
1400506,18219074,68f1a67b-cb28-4301-95a5-83304ebf8e6b,18011,trigger,1515915625467997301,email,2021-05-04,2021-05-04 09:08:17,f,NaT,9,manha,Tuesday,NaT,NaN,silencioso


# Split Dataset

## Contexto

In [12]:
cols_context = ['id', 'message_id', 'campaign_id', 'client_id', 'message_type', 'channel', 'sent_at', 'sending_time', 
                'day_of_week', 'last_click', 'recency_last_click', 'behavioral_segment']
df_contx = df_msg[cols_context]
df_contx.head()

,id,message_id,campaign_id,client_id,message_type,channel,sent_at,sending_time,day_of_week,last_click,recency_last_click,behavioral_segment
13115,18006688,7c48ccee-e2a6-4b50-a5d3-7c1dc8016f12,18011,1515915625467997301,trigger,email,2021-04-30 09:00:18,9,Friday,NaT,NaN,silencioso
1160263,18006690,a83c435e-4d05-4a1e-af01-9d67913f371a,18011,1515915625467997301,trigger,email,2021-04-30 13:06:09,13,Friday,NaT,NaN,silencioso
1160262,18006689,fbb0af74-6330-4fc8-a501-06abcc52e257,18010,1515915625467997301,trigger,email,2021-04-30 16:07:05,16,Friday,NaT,NaN,silencioso
1400507,18219075,d533bea7-63d7-4330-a73a-c1ad0b5f876c,19017,1515915625467997301,trigger,email,2021-05-04 08:27:03,8,Tuesday,NaT,NaN,silencioso
1400506,18219074,68f1a67b-cb28-4301-95a5-83304ebf8e6b,18011,1515915625467997301,trigger,email,2021-05-04 09:08:17,9,Tuesday,NaT,NaN,silencioso


In [13]:
cols_cat = ['message_type', 'channel', 'day_of_week', 'behavioral_segment']

oh_enc =  OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cols_cat_encoder = oh_enc.fit_transform(df_contx[cols_cat])
df_encoded = pd.DataFrame(cols_cat_encoder, columns=oh_enc.get_feature_names_out(cols_cat))

df_contx_enc = pd.concat([df_contx.drop(columns=cols_cat).reset_index(drop=True), df_encoded], axis=1)

df_contx_enc.head()

,id,message_id,campaign_id,client_id,sent_at,sending_time,last_click,recency_last_click,message_type_bulk,message_type_transactional,...,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,behavioral_segment_engajador,behavioral_segment_ocasional,behavioral_segment_silencioso
0,18006688,7c48ccee-e2a6-4b50-a5d3-7c1dc8016f12,18011,1515915625467997301,2021-04-30 09:00:18,9,NaT,NaN,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,18006690,a83c435e-4d05-4a1e-af01-9d67913f371a,18011,1515915625467997301,2021-04-30 13:06:09,13,NaT,NaN,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,18006689,fbb0af74-6330-4fc8-a501-06abcc52e257,18010,1515915625467997301,2021-04-30 16:07:05,16,NaT,NaN,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,18219075,d533bea7-63d7-4330-a73a-c1ad0b5f876c,19017,1515915625467997301,2021-05-04 08:27:03,8,NaT,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,18219074,68f1a67b-cb28-4301-95a5-83304ebf8e6b,18011,1515915625467997301,2021-05-04 09:08:17,9,NaT,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [14]:
df_contx_enc.to_csv('../dataset/processed/df_contexto.csv', index=False)
joblib.dump(oh_enc, '../dataset/processed/encoder_cols_context.pkl')

['../dataset/processed/encoder_cols_context.pkl']

## Recompensa

In [15]:
cols_reward = ['id', 'message_id', 'campaign_id', 'message_type', 'client_id', 'channel', 'sent_at', 'is_clicked']

df_reward = df_msg[cols_reward]
df_reward["reward"] = df_reward["is_clicked"].map({"t": 1, "f": 0})
df_reward.drop(columns=['is_clicked'],inplace=True)
df_reward.head()

C:\Users\Carlos\AppData\Local\Temp\ipykernel_10756\2858168929.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reward["reward"] = df_reward["is_clicked"].map({"t": 1, "f": 0})
C:\Users\Carlos\AppData\Local\Temp\ipykernel_10756\2858168929.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reward.drop(columns=['is_clicked'],inplace=True)


,id,message_id,campaign_id,message_type,client_id,channel,sent_at,reward
13115,18006688,7c48ccee-e2a6-4b50-a5d3-7c1dc8016f12,18011,trigger,1515915625467997301,email,2021-04-30 09:00:18,0
1160263,18006690,a83c435e-4d05-4a1e-af01-9d67913f371a,18011,trigger,1515915625467997301,email,2021-04-30 13:06:09,0
1160262,18006689,fbb0af74-6330-4fc8-a501-06abcc52e257,18010,trigger,1515915625467997301,email,2021-04-30 16:07:05,0
1400507,18219075,d533bea7-63d7-4330-a73a-c1ad0b5f876c,19017,trigger,1515915625467997301,email,2021-05-04 08:27:03,0
1400506,18219074,68f1a67b-cb28-4301-95a5-83304ebf8e6b,18011,trigger,1515915625467997301,email,2021-05-04 09:08:17,0


In [16]:
df_reward.to_csv('../dataset/processed/df_reward.csv', index=False)

## Ação

In [17]:
cols_action = ['id', 'message_id', 'campaign_id', 'message_type', 'client_id', 'channel', 'sent_at', 'period_day']

le_action = LabelEncoder()

df_action = df_msg[cols_action]
df_action["action"] = le_action.fit_transform(df_action["period_day"])
df_action.head()

C:\Users\Carlos\AppData\Local\Temp\ipykernel_10756\3100035259.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_action["action"] = le_action.fit_transform(df_action["period_day"])


,id,message_id,campaign_id,message_type,client_id,channel,sent_at,period_day,action
13115,18006688,7c48ccee-e2a6-4b50-a5d3-7c1dc8016f12,18011,trigger,1515915625467997301,email,2021-04-30 09:00:18,manha,1
1160263,18006690,a83c435e-4d05-4a1e-af01-9d67913f371a,18011,trigger,1515915625467997301,email,2021-04-30 13:06:09,tarde,3
1160262,18006689,fbb0af74-6330-4fc8-a501-06abcc52e257,18010,trigger,1515915625467997301,email,2021-04-30 16:07:05,tarde,3
1400507,18219075,d533bea7-63d7-4330-a73a-c1ad0b5f876c,19017,trigger,1515915625467997301,email,2021-05-04 08:27:03,manha,1
1400506,18219074,68f1a67b-cb28-4301-95a5-83304ebf8e6b,18011,trigger,1515915625467997301,email,2021-05-04 09:08:17,manha,1


In [18]:
df_action.to_csv('../dataset/processed/df_action.csv', index=False)